# Student Details

***

**Name:** Hoai Nhan Nguyen <br>
**Student Number:** sba24098 <br>
**Course:** Higher Diploma in Science in Artificial Intelligence Applications

***

# Data Cleaning and Transformation


**Importing Apache Spark Libraries.**

In [12]:
# Importing libraries 
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col, sum, when, regexp_replace, round

**Creating a new Spark Session.**

In [13]:
# Creating new SparkSession
spark = SparkSession.builder \
    .appName("MySparkApp") \
    .master("local[*]") \
    .config("spark.driver.host", "localhost") \
    .getOrCreate()

**Reading the Amazon-Products.csv in Hadoop.**

In [14]:
# Reading Amazon-Products.csv in Hadoop while applying options to read it correctly
df = spark.read.option("header", "true") \
               .option("inferSchema", "true") \
               .option("multiLine", "true") \
               .option("escape", "\"") \
               .option("quote", "\"") \
               .csv("hdfs://localhost:9000/user1/big_data_ca1/data/Amazon-Products.csv")

**Understanding the structure of the Spark Dataframe.**

In [15]:
# Reviewing the schema of the Spark Dataframe
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- main_category: string (nullable = true)
 |-- sub_category: string (nullable = true)
 |-- image: string (nullable = true)
 |-- link: string (nullable = true)
 |-- ratings: string (nullable = true)
 |-- no_of_ratings: string (nullable = true)
 |-- discount_price: string (nullable = true)
 |-- actual_price: string (nullable = true)



In [16]:
# Dropping the columns that are not required for this task 
df = df.drop("_c0","image","link")
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- main_category: string (nullable = true)
 |-- sub_category: string (nullable = true)
 |-- ratings: string (nullable = true)
 |-- no_of_ratings: string (nullable = true)
 |-- discount_price: string (nullable = true)
 |-- actual_price: string (nullable = true)



In [17]:
# Reviewing the rows of the Spark dataframe 
df.show()

+--------------------+-------------+----------------+-------+-------------+--------------+------------+
|                name|main_category|    sub_category|ratings|no_of_ratings|discount_price|actual_price|
+--------------------+-------------+----------------+-------+-------------+--------------+------------+
|Lloyd 1.5 Ton 3 S...|   appliances|Air Conditioners|    4.2|        2,255|       ₹32,999|     ₹58,990|
|LG 1.5 Ton 5 Star...|   appliances|Air Conditioners|    4.2|        2,948|       ₹46,490|     ₹75,990|
|LG 1 Ton 4 Star A...|   appliances|Air Conditioners|    4.2|        1,206|       ₹34,490|     ₹61,990|
|LG 1.5 Ton 3 Star...|   appliances|Air Conditioners|    4.0|           69|       ₹37,990|     ₹68,990|
|Carrier 1.5 Ton 3...|   appliances|Air Conditioners|    4.1|          630|       ₹34,490|     ₹67,790|
|Voltas 1.4 Ton 3 ...|   appliances|Air Conditioners|    4.0|        1,666|       ₹31,990|     ₹70,990|
|Lloyd 1.0 Ton 3 S...|   appliances|Air Conditioners|    4.2|   

25/04/17 20:33:39 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , name, main_category, sub_category, image, link, ratings, no_of_ratings, discount_price, actual_price
 Schema: _c0, name, main_category, sub_category, image, link, ratings, no_of_ratings, discount_price, actual_price
Expected: _c0 but found: 
CSV file: hdfs://localhost:9000/user1/big_data_ca1/data/Amazon-Products.csv


In [18]:
# Get the number of rows
num_rows = df.count()
# Get the number of columns
num_columns = len(df.columns)

# Printing the shape (rows, columns)
print(f"Shape of the DataFrame: ({num_rows}, {num_columns})")


25/04/17 20:33:41 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , name, main_category, sub_category, image, link, ratings, no_of_ratings, discount_price, actual_price
 Schema: _c0, name, main_category, sub_category, image, link, ratings, no_of_ratings, discount_price, actual_price
Expected: _c0 but found: 
CSV file: hdfs://localhost:9000/user1/big_data_ca1/data/Amazon-Products.csv
[Stage 14:>                                                         (0 + 1) / 1]

Shape of the DataFrame: (551585, 7)


**Checking the null values in the Spark Dataframe.**

In [21]:
# Counting the null or empty values per column
df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

25/04/17 20:34:22 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , name, main_category, sub_category, image, link, ratings, no_of_ratings, discount_price, actual_price
 Schema: _c0, name, main_category, sub_category, image, link, ratings, no_of_ratings, discount_price, actual_price
Expected: _c0 but found: 
CSV file: hdfs://localhost:9000/user1/big_data_ca1/data/Amazon-Products.csv
[Stage 23:>                                                         (0 + 1) / 1]

+----+-------------+------------+-------+-------------+--------------+------------+
|name|main_category|sub_category|ratings|no_of_ratings|discount_price|actual_price|
+----+-------------+------------+-------+-------------+--------------+------------+
|   0|            0|           0| 175794|       175794|         61163|       17813|
+----+-------------+------------+-------+-------------+--------------+------------+



**Handling Null values in the Spark Dataframe.**

In [ ]:
# Filtering out rows where both 'discount_price' and 'actual_price' are null.
df_clean = df.filter(~(col("discount_price").isNull() & col("actual_price").isNull()))

# Filling the null values for ratings, no_of_ratings, discount_price and actual_price to 0
df_clean = df_clean.fillna({"ratings": 0,"no_of_ratings": 0, "discount_price":0, "actual_price":0})

In [ ]:
# Counting the null or empty values per column
df_clean.select([
    sum(when(col(column_name).isNull() | (col(column_name) == ""), 1).otherwise(0)).alias(column_name + "_nulls")
    for column_name in df.columns
]).show()

**Handling duplicate rows in the Spark Dataframe.**

In [ ]:
# Removing the duplicate rows from the Spark DataFrame 
df_clean = df_clean.dropDuplicates()

**Cleaning the ratings and no_of_rating columns.**

In [ ]:
# Ensuring the rows where the 'ratings' column contains valid numbers (integers or decimals).
df_clean = df_clean.filter(F.col('ratings').rlike(r'^[0-9]*\.?[0-9]+$'))

# Ensuring ratings are between 0 and 5.0
df_clean = df_clean.filter((F.col('ratings') >= 0) & (F.col('ratings') <= 5.0))


In [ ]:
# Removing commas from 'no_of_ratings'
df_clean = df_clean.withColumn("no_of_ratings", regexp_replace(col("no_of_ratings"), ",", ""))

# Ensuring the rows where the 'no_of_ratings' column contains valid numbers (integers).
df_clean = df_clean.filter(col("no_of_ratings").rlike("^[0-9]+$"))

**Converting currency from Indian Rupee to Euro for the actual_price and discount_price columns.**

In [ ]:
# Removing ₹, commas and convert to double
df_converted = df_clean.withColumn(
    "actual_price",
    regexp_replace(col("actual_price"), "[₹,]", "").cast("double")  
)

# Converting INR to EUR (using conversion rate: 1 INR = 0.011 EUR)
conversion_rate = 0.011
df_converted = df_converted.withColumn(
    "actual_price",
    round(col("actual_price") * conversion_rate, 2)
)

In [ ]:
# Removing ₹, commas and convert to double
df_converted = df_converted.withColumn(
    "discount_price",
    regexp_replace(col("discount_price"), "[₹,]", "").cast("double") 
)

# Converting INR to EUR (using conversion rate: 1 INR = 0.011 EUR)
conversion_rate = 0.011
df_converted = df_converted.withColumn(
    "discount_price",
    round(col("discount_price") * conversion_rate, 2) 
)

**Saving converted Spark Dataframe as a CSV file in Hadoop**

In [ ]:
df_converted.write \
  .option("header", "true") \
  .option("quoteAll", "true") \
  .option("escape", "\"") \
  .csv("hdfs://localhost:9000/user1/big_data_ca1/data/Amazon-Products-Cleaned.csv")

# Inserting CSV Data into HBase 

**Importing HappyBase Library to connect to HBase.**

In [ ]:
import happybase

**Reading the Amazon-Products-Cleaned.csv in Hadoop**

In [9]:
# Reading Amazon-Products-Cleaned.csv in Hadoop while applying options to read it correctly
df = spark.read.option("header", "true") \
               .option("inferSchema", "true") \
               .option("multiLine", "true") \
               .option("escape", "\"") \
               .option("quote", "\"") \
               .csv("hdfs://localhost:9000/user1/big_data_ca1/data/Amazon-Products-Cleaned.csv")

**Reviewing the Schema of Amazon-Products-Cleaned.csv**

In [10]:
# Reviewing the schema of the Spark Dataframe
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- main_category: string (nullable = true)
 |-- sub_category: string (nullable = true)
 |-- ratings: double (nullable = true)
 |-- no_of_ratings: integer (nullable = true)
 |-- discount_price: double (nullable = true)
 |-- actual_price: double (nullable = true)



In [ ]:
df.show()

**Connecting to Hbase**

In [ ]:
# Connecting to HBase with the happybase library 
connection = happybase.Connection('localhost')
connection.open()

**Creating and adding data to the table.**

In [ ]:
# Defining the column families 
column_families = {
    'Item_Info': dict(),
    'Ratings_Info': dict(),
    'Pricing_Info': dict(),
}

# Creating the table amazon_products if it doesn't exist
table_name = 'amazon_products'
if table_name not in connection.tables():
    connection.create_table(table_name, column_families)

# Getting the table amazon_products
table = connection.table(table_name)

# For loop to add the data from Amazon-Products-Cleaned.csv to the table amazon_products
for idx, row in enumerate(df.rdd.collect(), start=1):
    # Generating a 6-character row key for each item
    row_key = str(idx).zfill(6)
    
    # Adding the data to the table
    table.put(row_key, {
        'Item_Info:name': row['name'],
        'Item_Info:main_category': row['main_category'],
        'Item_Info:sub_category': row['sub_category'],
        'Ratings_Info:ratings': str(row['ratings']),
        'Ratings_Info:no_of_ratings': str(row['no_of_ratings']),
        'Pricing_Info:discount_price': str(row['discount_price']),
        'Pricing_Info:actual_price': str(row['actual_price'])
    })


**Closing the connection**

In [ ]:
connection.close()

# Apache Spark - Basic Analysis and Insights 

In [22]:
# Reading Amazon-Products-Cleaned.csv in Hadoop while applying options to read it correctly
df = spark.read.option("header", "true") \
               .option("inferSchema", "true") \
               .option("multiLine", "true") \
               .option("escape", "\"") \
               .option("quote", "\"") \
               .csv("hdfs://localhost:9000/user1/big_data_ca1/data/Amazon-Products-Cleaned.csv")

In [76]:
# Filtering dataframe for the top 10 for ratings equal or more than 4.5 and number of ratings more than 1000.
df.filter((F.col("ratings") >= 4.5) & (F.col("no_of_ratings") > 1000)) \
  .orderBy(F.desc("ratings"), F.desc("no_of_ratings")) \
  .limit(10) \
  .show()

+--------------------+--------------------+--------------------+-------+-------------+--------------+------------+
|                name|       main_category|        sub_category|ratings|no_of_ratings|discount_price|actual_price|
+--------------------+--------------------+--------------------+-------+-------------+--------------+------------+
|Pampers Swaddlers...|toys & baby products|             Diapers|    4.9|        26160|        143.09|      204.48|
|Medela Breastmilk...|toys & baby products|   Nursing & Feeding|    4.9|         7404|         52.95|       133.1|
|Pampers Diapers S...|toys & baby products|             Diapers|    4.9|         6553|        220.71|       607.2|
|DOWAN multi color...|      home & kitchen|    Kitchen & Dining|    4.9|         5485|          4.94|       54.99|
|Scrub Daddy Insta...|          appliances|Kitchen & Home Ap...|    4.9|         2285|         15.39|        38.5|
|Scrub Daddy Insta...|          appliances|      All Appliances|    4.9|        

In [84]:
# Filtering dataframe for the bottom 10 for ratings equal or more than 1 and number of ratings more than 1000.
df.filter((F.col("ratings") >= 1) & (F.col("no_of_ratings") > 1000)) \
  .orderBy(F.asc("ratings"), F.asc("no_of_ratings")) \
  .limit(10) \
  .show()

+--------------------+-------------------+--------------------+-------+-------------+--------------+------------+
|                name|      main_category|        sub_category|ratings|no_of_ratings|discount_price|actual_price|
+--------------------+-------------------+--------------------+-------+-------------+--------------+------------+
|DEVCOMM Phone Blu...|tv, audio & cameras|     All Electronics|    2.5|         1225|          1.54|         3.3|
|Prestige Roti Mak...|         appliances|      All Appliances|    2.6|         1411|         31.61|       37.35|
|Prestige Roti Mak...|         appliances|Kitchen & Home Ap...|    2.6|         1411|         31.61|       37.35|
|TYING Men's Trend...|        men's shoes|        Casual Shoes|    2.7|         1942|          3.29|       10.98|
|Kuber Industries ...|     home & kitchen|  All Home & Kitchen|    2.7|         1977|          3.83|        5.49|
|Generic Silica Ge...|    beauty & health|             Make-up|    2.8|         1645|   

In [83]:
# Grouping by 'main_category' and calculate summary statistics for prices
df_price_stats = df.groupBy("main_category").agg(
    F.format_number(F.min("actual_price"), 2).alias("min_actual_price"),
    F.format_number(F.max("actual_price"), 2).alias("max_actual_price"),
    F.format_number(F.avg("actual_price"), 2).alias("avg_actual_price"),
    F.format_number(F.min("discount_price"), 2).alias("min_discount_price"),
    F.format_number(F.max("discount_price"), 2).alias("max_discount_price"),
    F.format_number(F.avg("discount_price"), 2).alias("avg_discount_price"),
)

# Show dataframe
df_price_stats.show()

+--------------------+----------------+----------------+----------------+------------------+------------------+------------------+
|       main_category|min_actual_price|max_actual_price|avg_actual_price|min_discount_price|max_discount_price|avg_discount_price|
+--------------------+----------------+----------------+----------------+------------------+------------------+------------------+
|    women's clothing|            0.65|          549.99|           19.39|              0.00|            252.99|              6.69|
|         men's shoes|            0.43|          734.57|           42.23|              0.00|            733.14|             16.92|
|toys & baby products|            0.16|          630.27|           20.80|              0.00|            332.56|              9.58|
| home, kitchen, pets|           22.00|           87.56|           52.14|             19.80|             55.44|             34.89|
|          appliances|            0.65|        6,600.00|           90.30|          

In [51]:
# Calculating the total discount loss based on actual price - discount price
df.withColumn("discount_loss", F.col("actual_price") - F.col("discount_price")) \
  .agg(
      F.format_number(F.sum("actual_price"), 2).alias("total_actual_price"),
      F.format_number(F.sum("discount_price"), 2).alias("total_discount_price"),
      F.format_number(F.sum("discount_loss"), 2).alias("total_discount_loss")
  ) \
  .show()

+------------------+--------------------+-------------------+
|total_actual_price|total_discount_price|total_discount_loss|
+------------------+--------------------+-------------------+
|    133,376,837.61|       12,885,616.45|     120,491,221.16|
+------------------+--------------------+-------------------+



In [64]:
# Grouping by 'main_category' and calculate average rating and total number of ratings
df.groupBy("main_category") \
  .agg(
      F.round(F.avg("ratings"), 1).alias("average_rating"),
      F.sum("no_of_ratings").alias("total_number_ratings")
  ) \
  .orderBy("total_number_ratings") \
  .show()

+--------------------+--------------+--------------------+
|       main_category|average_rating|total_number_ratings|
+--------------------+--------------+--------------------+
| home, kitchen, pets|           0.0|                   0|
|               music|           3.2|              283319|
|       women's shoes|           2.0|              547897|
|        pet supplies|           3.6|              959921|
| industrial supplies|           2.8|             1050115|
|      bags & luggage|           1.6|             1365586|
|     car & motorbike|           3.0|             1498404|
|grocery & gourmet...|           3.7|             1635936|
|       kids' fashion|           1.9|             1679459|
|    sports & fitness|           2.8|             2784355|
|      men's clothing|           2.2|             5093413|
|         men's shoes|           2.0|             5805636|
|      home & kitchen|           3.7|             9319642|
|     beauty & health|           3.2|            1021385